In [11]:
import os
import random
import shutil
from PIL import Image
import torchvision.transforms as transforms

In [21]:
import os
import random
import shutil
from PIL import Image
from torchvision import transforms

In [23]:


def balance_dataset(train_dir, target_size=4000):
    """
    Balance the dataset by:
    1. Randomly removing excess images from overrepresented classes
    2. Augmenting underrepresented classes
    """
    # Get list of subdirectories (emotion categories)
    categories = [category for category in os.listdir(train_dir)
                  if os.path.isdir(os.path.join(train_dir, category))]
    
    # Count images in each category
    image_counts = {category: len(os.listdir(os.path.join(train_dir, category))) for category in categories}
    
    # Print initial image counts
    print("Initial Image Counts:")
    for category, count in image_counts.items():
        print(f"{category}: {count} images")
    
    # Define augmentation transformations
    transform = transforms.Compose([
        transforms.RandomRotation(10),           # Random rotation
        transforms.RandomHorizontalFlip(),       # Random horizontal flip
        transforms.RandomVerticalFlip(),         # Random vertical flip
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Random color jitter
        transforms.RandomAffine(degrees=5, translate=(0.1, 0.1))  # Random affine transform
    ])
    
    # Create a backup of the original dataset
    backup_dir = train_dir + "_backup"
    shutil.copytree(train_dir, backup_dir, dirs_exist_ok=True)
    
    # Process each category
    for category in categories:
        category_path = os.path.join(train_dir, category)
        images = os.listdir(category_path)
        
        if len(images) > target_size:
            # If category has more than target size, randomly remove excess images
            excess_images = random.sample(images, len(images) - target_size)
            for img in excess_images:
                os.remove(os.path.join(category_path, img))
            print(f"Removed {len(excess_images)} images from {category}")
        
        elif len(images) < target_size:
            # If category has fewer than target size, augment images
            augmented_images_needed = target_size - len(images)
            augmented_count = 0
            
            while augmented_count < augmented_images_needed:
                # Randomly select an image to augment
                source_image = random.choice(images)
                source_path = os.path.join(category_path, source_image)
                
                # Open the image
                img = Image.open(source_path)
                
                # Apply transformations
                augmented_img = transform(img)
                
                # Generate a unique filename
                base_name = os.path.splitext(source_image)[0]
                augmented_img_name = f"{base_name}_aug_{augmented_count}.jpg"
                augmented_img_path = os.path.join(category_path, augmented_img_name)
                
                # Save the augmented image
                augmented_img.save(augmented_img_path)
                augmented_count += 1
            
            print(f"Augmented {augmented_count} images for {category}")
    
    # Verify final counts
    final_counts = {category: len(os.listdir(os.path.join(train_dir, category))) for category in categories}
    print("\nFinal Image Counts:")
    for category, count in final_counts.items():
        print(f"{category}: {count} images")


# Set the path to your image dataset
train_dir = r"dataset\images\train"  # Use a raw string to avoid escape sequence errors

# Balance the dataset
balance_dataset(train_dir)


Initial Image Counts:
angry: 3993 images
disgust: 436 images
fear: 4103 images
happy: 7164 images
neutral: 4982 images
sad: 4938 images
surprise: 3205 images
Augmented 7 images for angry
Augmented 3564 images for disgust
Removed 103 images from fear
Removed 3164 images from happy
Removed 982 images from neutral
Removed 938 images from sad
Augmented 795 images for surprise

Final Image Counts:
angry: 4000 images
disgust: 4000 images
fear: 4000 images
happy: 4000 images
neutral: 4000 images
sad: 4000 images
surprise: 4000 images


In [25]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11s-cls.yaml")  # build a new model from YAML
model = YOLO("yolo11s-cls.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo11s-cls.yaml").load("yolo11s-cls.pt")  # build from YAML and transfer weights

# Train the model
results = model.train(data=r"dataset\images", epochs=50, imgsz=64)

YOLO11s-cls summary: 151 layers, 5,545,488 parameters, 5,545,488 gradients, 12.2 GFLOPs


100%|██████████| 13.0M/13.0M [00:00<00:00, 82.9MB/s]


YOLO11s-cls summary: 151 layers, 5,545,488 parameters, 5,545,488 gradients, 12.2 GFLOPs
Transferred 234/236 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.99 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.56  Python-3.12.9 torch-2.5.1+cpu CPU (AMD Ryzen Threadripper 3960X 24-Core Processor)
engine\trainer: task=classify, mode=train, model=yolo11s-cls.yaml, data=dataset\images, epochs=50, time=None, patience=100, batch=16, imgsz=64, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train12, exist_ok=False, pretrained=yolo11s-cls.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, sourc

train: Scanning C:\work\alma\Facial-Emotion-Recognition\dataset\images\train... 28000 images, 0 corrupt: 100%|██████████| 28000/28000 [00:00<?, ?it/s]
val: Scanning C:\work\alma\Facial-Emotion-Recognition\dataset\images\test... 11929 images, 1 corrupt: 100%|██████████| 11929/11929 [00:00<?, ?it/s]

val: WARNING  C:\work\alma\Facial-Emotion-Recognition\dataset\images\test\disgust\21744_aug_1823.jpg: ignoring corrupt image/label: cannot identify image file 'C:\\work\\alma\\Facial-Emotion-Recognition\\dataset\\images\\test\\disgust\\21744_aug_1823.jpg'
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
Image sizes 64 train, 64 val
Using 0 dataloader workers
Logging results to runs\classify\train12
Starting training for 50 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

      50/50         0G     0.7961         16         64: 100%|██████████| 1750/1750 [02:42<00:00, 10.77it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 373/373 [00:14<00:00, 26.02it/s]

                   all      0.559      0.986



50 epochs completed in 2.702 hours.
Optimizer stripped from runs\classify\train12\weights\last.pt, 11.0MB
Optimizer stripped from runs\classify\train12\weights\best.pt, 11.0MB

Validating runs\classify\train12\weights\best.pt...
Ultralytics 8.3.56  Python-3.12.9 torch-2.5.1+cpu CPU (AMD Ryzen Threadripper 3960X 24-Core Processor)
YOLO11s-cls summary (fused): 112 layers, 5,443,095 parameters, 0 gradients, 12.0 GFLOPs
WARNING  Dataset 'split=val' not found, using 'split=test' instead.
train: C:\work\alma\Facial-Emotion-Recognition\dataset\images\train... found 28000 images in 7 classes  
val: None...
test: C:\work\alma\Facial-Emotion-Recognition\dataset\images\test... found 11930 images in 7 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 373/373 [00:13<00:00, 27.85it/s]


                   all       0.56      0.986
Speed: 0.0ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train12


In [5]:
from ultralytics import YOLO
model = YOLO("yolo11s_emotion.pt")
results = model("unseen_data/happy_pramod.jpg")
results[0].show()


image 1/1 /mnt/c/work/alma/Facial-Emotion-Recognition/unseen_data/happy_pramod.jpg: 224x224 happy 0.83, neutral 0.07, sad 0.06, angry 0.03, fear 0.00, 6.4ms
Speed: 29.1ms preprocess, 6.4ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)


Try running the update-desktop-database command. If you
don't have this command you should install the
desktop-file-utils package. This package is available from
http://freedesktop.org/wiki/Software/desktop-file-utils/
No applications found for mimetype: image/png
./usr/bin/xdg-open: 882: x-www-browser: not found
/usr/bin/xdg-open: 882: firefox: not found
/usr/bin/xdg-open: 882: iceweasel: not found
/usr/bin/xdg-open: 882: seamonkey: not found
/usr/bin/xdg-open: 882: mozilla: not found
/usr/bin/xdg-open: 882: epiphany: not found
/usr/bin/xdg-open: 882: konqueror: not found
/usr/bin/xdg-open: 882: chromium: not found
/usr/bin/xdg-open: 882: chromium-browser: not found
/usr/bin/xdg-open: 882: google-chrome: not found
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method avai